# Simulated Quantum Annealing Solver 

----

###### Note:
If you viewing this notebook directly at github.com, the LaTeX formulas may not be rendered correctly. 
Thus, we highly recommend to use [nbviewer](https://nbviewer.jupyter.org/) for this purpose. 

You can view this notebook directly at: https://nbviewer.jupyter.org/github/quantum-circle/qc_qubosolv/blob/master/examples/example.ipynb

---

<br>

## A Minimal Example

In order to solve a QUBO, we can use main incubator's high performance Simulated (Quantum) Annealing solver to estimate the bitstring $\mathbf{x}$, that minimizes the objective function. To do so, we simply need to:

1. Intialize the solver client `qc_qubosolv.Solver` with our Quantum Circle Developer credentials.
2. Submit the `qubo_matrix` and `qubo_offset` objects via the solver's API and select an appropriate algorithm, e.g. "sqa" for Simulated Quantum Annealing.

In [1]:
from qc_qubosolv import Solver


# 1. Create the QUBO matrix to be solved
qubo_matrix = [
    [-1, 2], 
    [2, -4]
]

# 2. Initialize the solver client
solver = Solver(
    username='YOUR_USERNAME_HERE', 
    password='YOUR_PASSWORD_HERE'
)

# 3. Solve the previously created QUBO problem
result = solver.solve(qubo_matrix, algorithm='sqa')

# 4. Print the optimal bitstring and its corresponding objective function value
print(f"Optimal bitstring: x = {result.optimal_bitstring}")
print(f"Optimal energy: f(x) = {result.optimal_energy}")

Optimal bitstring: x = [0 1]
Optimal energy: f(x) = -4.0


## Generate and Solve QUBO Models

#### Note
For this example, you will need to have Quantum Circle's QUBO generator module `qc_qubogen` installed.


### Generate a QUBO
To illustrate a slightly more advanced example, we choose to solve the optimization problem given by Glover et. al.$^{[1]}$:

$$ f(\mathbf{x}) = -5x_0 -3x_1 -8x_2 -6x_3 + 4x_0x_1 + 8x_0x_2 + 2x_1x_2 + 10x_2x_3 \; ,$$

which will be converted to the QUBO problem:

$$ f(\mathbf{x})  = \left(\begin{matrix} x_0 & x_1 & x_3 & x_4 \end{matrix}\right) \left(\begin{matrix} -5 & 2 & 4 & 0 \\ 2 & -3 & 1 & 0 \\ 4 & 1 & -8 & 5 \\ 0 & 0 & 5 & -6 \end{matrix} \right) \left(\begin{matrix} x_0\\ x_1 \\ x_2 \\ x_3 \end{matrix}\right) $$

Note that we omit the constant offset $c$ here, since $c = 0$.

<br>

Here, we're going to use the `qc_qubogen` module's QUBO generator `QUBOModel` and helper function `qc_qubogen.get_symbolic_binary_variables(n=4)` .

---

[1] Glover, F., Kochenberger, G. and Du, Yu. Quantum Bridge Analytics I: a tutorial on formulating and using QUBO models. *Available at:* https://arxiv.org/pdf/1811.11538.pdf

In [2]:
from qc_qubogen import QUBOModel, get_symbolic_binary_variables


# 1. Create a list of symbolic binary variables (sympy.Symbol)
x = get_symbolic_binary_variables(n=4)

# 2. Define the objective function to be minimized
objective = - 5*x[0] - 3*x[1] - 8*x[2] - 6*x[3] + 4*x[0]*x[1] + 8*x[0]*x[2] + 2*x[1]*x[2] + 10*x[2]*x[3]

# 3. Initialize the `QUBOModel` using the created objective function
model = QUBOModel(objective)

# 4. Convert the problem and return the QUBO matrix and offset constant
qubo_matrix, qubo_offset = model.model_to_qubo(return_offset=True)

print(f"QUBO matrix: Q = \n{qubo_matrix}")

QUBO matrix: Q = 
[[-5.  2.  4.  0.]
 [ 2. -3.  1.  0.]
 [ 4.  1. -8.  5.]
 [ 0.  0.  5. -6.]]


## Solve the QUBO

In [3]:
from qc_qubosolv import Solver


solver = Solver(
    username='YOUR_USERNAME_HERE', 
    password='YOUR_PASSWORD_HERE'
)

result = solver.solve(qubo_matrix, qubo_offset, algorithm='sqa')

print(f"Optimal bitstring: x = {result.optimal_bitstring}")
print(f"Optimal energy: f(x) = {result.optimal_energy}")

Optimal bitstring: x = [1 0 0 1]
Optimal energy: f(x) = -11.0
